In [1]:
import numpy as np
from simulation_framework.item_response_simulation import item_response_simulation
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma
import scipy
from simulation_experiment import mirt_simulation_experiment
from analyse_experiment import get_result_df
from analyse_experiment import print_result_from_df

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models
C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


In [2]:
import datetime
from itertools import product

def repeat_mirt_experiment(experiment, repetitions, sample_sizes=[], item_dims=[], latent_dims=[], file=""):
    error_occured = False
    number_errors = 0
    errors = []
    multiple_result_dict = {}
    variant_list = list(product(sample_sizes, item_dims, latent_dims))
    if file != "":
        try:
            prior_results_df = pd.read_csv(file)
        except FileNotFoundError:
            prior_results_df = pd.DataFrame()
    else:
        prior_results_df = pd.DataFrame()
    experiment_df = prior_results_df.copy()
    for i in range(0, repetitions):
        for variant in variant_list:
            sample_size = variant[0]
            item_dimension = variant[1]
            latent_dimension = variant[2]
            if experiment_df.size > 0:
                prior_variant_results = experiment_df[(experiment_df["latent_dimension"] == latent_dimension) &
                                                    (experiment_df["item_dimension"] == item_dimension) &
                                                    (experiment_df["sample_size"] == sample_size)]
                if len(prior_variant_results["key"].unique()) < repetitions:
                    run_variant=True
                else:
                    run_variant=False
            else:
                run_variant=True
            if run_variant: 
                key = datetime.datetime.now()
                try:
                    multiple_result_dict[key] = experiment(sample_size=sample_size, 
                                                item_dimension=item_dimension, latent_dimension=latent_dimension)
                except Exception as e:
                    print("Exception occured")
                    print(e)
                    error_occured=True
                    number_errors += 1
                    errors.append(e)
                    continue
                variant_df = get_result_df({key: multiple_result_dict[key]})
                experiment_df = pd.concat((experiment_df, variant_df), axis=0)
                if file != "":
                    experiment_df.reset_index(drop=True)
                    experiment_df.to_csv(file, index=False)
    #if error_occured:
    #    pass #TODO Recursion for remaining variants
    return(experiment_df, errors)


## Experiment 0: MIRT-2PL-Gain Performance Benchmark

In [3]:
experiment_0 = lambda sample_size, item_dimension, latent_dimension: mirt_simulation_experiment(
                                             sample_size=sample_size, item_dimension=item_dimension, latent_dimension=latent_dimension, 
                                             q_type="full", stop_threshold=0.02, 
                                             early_person_method="BFGS",
                                             late_person_method="BFGS",
                                            sigma_constraint="early_constraint",
                                            methods=["late_em", "real_early", "pure_competency", "initial", "difference"])
performance_df, errors= repeat_mirt_experiment(experiment_0, repetitions=3, sample_sizes=[120], latent_dims=[2], item_dims=[20], file="results/test.csv")

In [4]:
print_result_from_df(performance_df, description="With additional constraint on A rotation and according actual A rotation", reference_method="late_em")

------------------------------------
##### Results for With additional constraint on A rotation and according actual A rotation
Latent dimension: 2.0,  Item dimension: 20.0, sample size 120.0 \
Early runtime: 4.98 seconds, 6.666666666666667 steps, 0.75 seconds per step \
Late runtime: 77.63 seconds, 10.0 steps, 7.76 seconds per step \
Early Marginal Likelihood, Optimal: -1174.04, Estimated: -1153.63, Initial -1279.9 \
Late Marginal Likelihood, Optimal: -1228.33, Estimated: -1210.62, Initial -1577.82
| method          |    early_A |   early_delta |   early_sigma |    late_A |   late_delta |   late_sigma |      psi |
|:----------------|-----------:|--------------:|--------------:|----------:|-------------:|-------------:|---------:|
| difference      |   0.746842 |      0.499456 |     0.0636632 |   1.86797 |     0.804013 |     0.141948 | 0.193611 |
| initial         |   1.80109  |      0.506282 |     0.245413  |   1.79464 |     1.16631  |     0.467849 | 0.400262 |
| late_em         |   0

In [5]:
performance_df[performance_df["method"] == "late_em"]["rmse_psi"]

0     0.423450
5     0.267160
10    0.297481
Name: rmse_psi, dtype: float64


------------------------------------
##### Results for Increasing Monte Carlo sample-size based on ttest with normal MC
Latent dimension: 2,  Item dimension: 6, sample size 100 \
Runtime: 3.3 seconds, 15 steps, 0.22 seconds per step \
Optimal marginal Likelihood: -371.46, Estimated: -365.03, Initial -374.57
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.660667 |     0.316008 |    0.0573464 |
| early_initial | 0.509902 |     0.28936  |    0.212132  |
| girth         | 1.0658   |     0.31617  |    0.12943   |
| early_direct  | 0.848957 |     0.59659  |    0.116858  |

------------------------------------
##### Results for Untested full Model
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 2.56 seconds, 8 steps, 0.32 seconds per step \
Late runtime: 107.71 seconds, 5 steps, 21.54 seconds per step \
Early Marginal Likelihood, Optimal: -470.21, Estimated: -463.83, Initial -527.4 \
Late Marginal Likelihood, Optimal: 3.89, Estimated: 3.92, Initial 3.81
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   1.08941 |      0.391419 |     0.0115966 |  1.83043 |      1.33621 |     0.674368 | 0.395368 |
| initial   |   3.07758 |      0.621379 |     0.329017  |  2.74743 |      1.52522 |     0.320885 | 0.210157 |
####
Performance on Individual Level \
rmse theta: 0.864016477222037 \
rmse gain: 0.8736126145557623

------------------------------------
##### Results for Fixed marginal Loglikelihood
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 2.5 seconds, 7 steps, 0.36 seconds per step \
Late runtime: 322.67 seconds, 9 steps, 35.85 seconds per step \
Early Marginal Likelihood, Optimal: -494.37, Estimated: -479.73, Initial -540.27 \
Late Marginal Likelihood, Optimal: -359.97, Estimated: -381.08, Initial -431.02
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   1.42983 |      0.837686 |     0.139583  |  4.08714 |      2.46488 |    0.243386  | 0.343469 |
| initial   |   2.6978  |      0.668599 |     0.0490025 |  5.45633 |      1.95892 |    0.0963787 | 0.22315  |
####
Performance on Individual Level \
rmse theta: 0.6755600631466341 \
rmse gain: 780.9161055994681

------------------------------------
##### Results for Genetic Algorithm
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.42 seconds, 7 steps, 0.49 seconds per step \
Late runtime: 166.44 seconds, 10 steps, 16.64 seconds per step \
Early Marginal Likelihood, Optimal: -445.8, Estimated: -440.04, Initial -525.38 \
Late Marginal Likelihood, Optimal: -452.07, Estimated: -436.89, Initial -535.83
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |   3.52076 |      0.390601 |     0.0378639 |  5.26244 |     0.403017 |    0.134472  | 0.317107 |
| initial   |   5.04367 |      0.567904 |     0.0948937 |  4.62289 |     1.22823  |    0.0472347 | 0.344357 |
####
Performance on Individual Level \
rmse theta: 0.8067692103719907 \
rmse gain: 560.8514011989563

------------------------------------
##### Results for With constrained BFGS
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.39 seconds, 8 steps, 0.42 seconds per step \
Late runtime: 222.08 seconds, 11 steps, 20.19 seconds per step \
Early Marginal Likelihood, Optimal: -450.38, Estimated: -444.2, Initial -522.62 \
Late Marginal Likelihood, Optimal: -469.55, Estimated: -464.8, Initial -539.91
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| estimated |  0.683488 |      0.604727 |     0.0795495 |  1.19288 |     0.397077 |    0.163978  | 0.367325 |
| initial   |  2.31601  |      0.712129 |     0.0905097 |  2.83422 |     1.04892  |    0.0775696 | 0.226698 |
####
Performance on Individual Level \
rmse theta: 0.7139781405467193 \
rmse gain: 0.8053860461431572

------------------------------------
##### Results for With unconstrained BFGS and save ICC
Latent dimension: 2,  Item dimension: 10, sample size 200 \
Early runtime: 4.94 seconds, 10 steps, 0.49 seconds per step \
Late runtime: 217.77 seconds, 14 steps, 15.56 seconds per step \
Early Marginal Likelihood, Optimal: -1025.93, Estimated: -1014.24, Initial -1143.43 \
Late Marginal Likelihood, Optimal: -1019.85, Estimated: -942.56, Initial -1165.24
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |     psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|--------:|
| estimated |   1.28713 |      0.405811 |      0.022486 |  2.34827 |      3.08274 |    0.0799031 | 0.25137 |
| initial   |   2.81054 |      0.442962 |      0.257387 |  4.62677 |      1.05851 |    0.0787717 | 0.33902 |
####
Performance on Individual Level \
rmse theta: 0.8966706033114552 \
rmse gain: 0.8351295524334317

------------------------------------
##### Results for With unconstrained BFGS and save ICC and density
Latent dimension: 2,  Item dimension: 10, sample size 100 \
Early runtime: 3.18 seconds, 7 steps, 0.45 seconds per step \
Late runtime: 138.53 seconds, 15 steps, 9.24 seconds per step \
Early Marginal Likelihood, Optimal: -520.2, Estimated: -510.43, Initial -543.26 \
Late Marginal Likelihood, Optimal: -423.14, Estimated: -395.86, Initial -478.51
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |       psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|----------:|
| estimated |   1.46673 |      0.222442 |      0.180454 |  8.89911 |      3.884   |    0.0590434 | 0.0987126 |
| initial   |   1.98992 |      0.373841 |      0.138664 |  3.10531 |      1.01749 |    0.0374059 | 0.279826  |
####
Performance on Individual Level \
rmse theta: 0.823547261347214 \
rmse gain: 1.1151668629591303

------------------------------------
##### Results for With unconstrained BFGS and unsave ICC and save density
Latent dimension: 2,  Item dimension: 12, sample size 100 \
Early runtime: 3.87 seconds, 8 steps, 0.48 seconds per step \
Late runtime: 134.42 seconds, 9 steps, 14.94 seconds per step \
Early Marginal Likelihood, Optimal: -602.6, Estimated: -592.22, Initial -670.75 \
Late Marginal Likelihood, Optimal: -456.88, Estimated: -452.65, Initial -554.35
|           |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |       psi |
|:----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|----------:|
| estimated |  0.690325 |      0.546951 |     0.0650538 |  1.27364 |     0.938354 |    0.0915703 | 0.0813927 |
| initial   |  2.45374  |      0.70644  |     0.213122  |  2.99672 |     1.01538  |    0.137037  | 0.289388  |
####
Performance on Individual Level \
rmse theta: 0.9424593402985434 \
rmse gain: 0.9094035651380942

------------------------------------
##### Results for With no constraint on early-conditional
Latent dimension: 2.0,  Item dimension: 12.0, sample size 130.0 \
Early runtime: 3.72 seconds, 8.2 steps, 0.45 seconds per step \
Late runtime: 80.16 seconds, 13.0 steps, 6.17 seconds per step \
Early Marginal Likelihood, Optimal: -774.22, Estimated: -764.32, Initial -866.47 \
Late Marginal Likelihood, Optimal: -1019.18, Estimated: -921.48, Initial -1086.68
| method     |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:-----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference |   1.34612 |      0.534403 |     0.0358224 |  0.92071 |     0.700088 |    0.0741752 | 0.239726 |
| initial    |   2.74906 |      0.642181 |     0.179478  |  1.75942 |     1.26116  |    0.455647  | 0.321535 |
| late_em    |   1.39325 |      0.590534 |     0.0421719 |  1.40764 |     0.688456 |    1.40617   | 0.861078 |
####
Performance on Individual Level \
rmse theta: 0.531318687109031 \
rmse gain: 0.9887035356487519

------------------------------------
##### Results for With additional constraint on A rotation
Latent dimension: 2.0,  Item dimension: 12.0, sample size 130.0 \
Early runtime: 4.39 seconds, 9.2 steps, 0.48 seconds per step \
Late runtime: 112.4 seconds, 16.8 steps, 6.69 seconds per step \
Early Marginal Likelihood, Optimal: -729.67, Estimated: -719.15, Initial -836.73 \
Late Marginal Likelihood, Optimal: -858.64, Estimated: -839.4, Initial -1061.49
| method     |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:-----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference |   1.7291  |      0.671763 |     0.0601324 |  1.45645 |     0.541567 |     0.129567 | 0.197979 |
| initial    |   3.80043 |      0.689331 |     0.208922  |  2.84149 |     1.49502  |     0.345074 | 0.266681 |
| late_em    |   1.75456 |      0.651589 |     0.0588596 |  2.541   |     1.51147  |     0.262643 | 0.255923 |
####
Performance on Individual Level \
rmse theta: 0.4893078017207201 \
rmse gain: 0.6258517067643191

------------------------------------
##### Results for With additional constraint on A rotation and according actual A rotation
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 3.53 seconds, 8.0 steps, 0.44 seconds per step \
Late runtime: 46.2 seconds, 10.2 steps, 4.53 seconds per step \
Early Marginal Likelihood, Optimal: -553.79, Estimated: -542.9, Initial -628.01 \
Late Marginal Likelihood, Optimal: -639.05, Estimated: -633.58, Initial -798.0
| method          |    early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:----------------|-----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference      |   0.897254 |      0.5579   |     0.0803415 |  1.6846  |      1.06954 |    0.0903535 | 0.246921 |
| initial         |   2.46837  |      0.50758  |     0.170017  |  3.08702 |      1.56427 |    0.386886  | 0.310015 |
| late_em         |   0.956962 |      0.569663 |     0.0870731 | 11.9947  |      1.7126  |    0.471784  | 0.332751 |
| pure_competency |   0.917059 |      0.600976 |     0.198952  |  1.90672 |      1.25602 |    0.699273  | 0.428938 |
| real_early      | nan        |    nan        |   nan         |  2.01509 |      1.56985 |    0.791628  | 0.480195 |
####
Performance on Individual Level \
rmse theta: 0.5140882904695336 \
rmse gain estimated: 0.6552893063329204 \
rmse gain predicted: 0.7734180679791622

## Experiment 1: MIRT-2PL Parameter Recovery

### Full Q-Matrix

In [6]:
experiment_0 = lambda sample_size, item_dimension, latent_dimension: mirt_simulation_experiment(
                                             sample_size=sample_size, item_dimension=item_dimension, latent_dimension=latent_dimension, 
                                             q_type="full", stop_threshold=0.01, 
                                             early_person_method="BFGS",
                                             late_person_method="BFGS",
                                            sigma_constraint="early_constraint",
                                            methods=["late_em", "real_early", "pure_competency", "initial", "difference"])
result_df, errors= repeat_mirt_experiment(experiment_0, repetitions=10, sample_sizes=[30, 100, 200], latent_dims=[2, 3], item_dims=[10, 20], file="results/full_q.csv")

Real latent covariance: [[1.     0.4646 0.5196 0.6241]
 [0.4646 1.     0.0328 0.5509]
 [0.5196 0.0328 0.7    0.2394]
 [0.6241 0.5509 0.2394 0.7819]]
Start late EM
Covariance matrix is good: True
EM Iteration 2
Current Monte Carlo Sample size: 300


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 8.709977114680198, current marginal loglikelihood: -159.62488094700905
EM Iteration 3
Current Monte Carlo Sample size: 300


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 3: current parameter_diff: 5.857886592718995, current marginal loglikelihood: -157.6179363498517
EM Iteration 4
Current Monte Carlo Sample size: 324


...

c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 5: current parameter_diff: 0.29990883995030837, current marginal loglikelihood: -1153.4051925069498
EM Iteration 6
Current Monte Carlo Sample size: 754


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 6: current parameter_diff: 1.1039580536643658, current marginal loglikelihood: -1150.4986399250934


KeyboardInterrupt: 

In [7]:
print_result_from_df(result_df, description="Additional convolution constraint")

NameError: name 'result_df' is not defined


------------------------------------
##### Results for Monte Carlo sample varying through ttest with normal MC
Latent dimension: 1,  Item dimension: 20, sample size 1000 \
Runtime: 37.86 seconds, 20 steps, 1.89 seconds per step \
Optimal marginal Likelihood: -8714.19, Estimated: -8697.21, Initial -9512.15
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.182642 |     0.211132 |            0 |
| early_initial | 1.69549  |     1.49975  |            0 |
| girth         | 0.313235 |     4.2295   |            0 |

------------------------------------
##### Results for Unsafe Q-Item
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 2.83 seconds, 6.666666666666667 steps, 0.42 seconds per step \
Late runtime: 89.06 seconds, 8.0 steps, 11.13 seconds per step \
Early Marginal Likelihood, Optimal: -588.95, Estimated: -577.35, Initial -651.28 \
Late Marginal Likelihood, Optimal: -503.73, Estimated: -493.19, Initial -594.28
| method   |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:---------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| initial  |  2.45062  |      0.659433 |     0.221324  |  4.21137 |      1.32515 |    0.0867149 | 0.329417 |
| late_em  |  0.962137 |      0.589822 |     0.0997021 |  2.94637 |      1.26479 |    0.119171  | 0.280201 |


------------------------------------
##### Results for Safe Q-Item with fixed Q-Matrix usage
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 3.61 seconds, 7.8 steps, 0.46 seconds per step \
Late runtime: 59.49 seconds, 10.4 steps, 5.72 seconds per step \
Early Marginal Likelihood, Optimal: -582.92, Estimated: -569.96, Initial -644.32 \
Late Marginal Likelihood, Optimal: -578.6, Estimated: -558.43, Initial -706.82
| method   |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:---------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| initial  |   2.88087 |      0.517633 |      0.182391 |  3.17101 |      1.1477  |     0.161164 | 0.26501  |
| late_em  |   1.16439 |      0.584869 |      0.112529 |  3.46094 |      2.04145 |     0.172379 | 0.268391 |
####

------------------------------------
##### Results for Additional real-early baseline
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 3.41 seconds, 7.6 steps, 0.45 seconds per step \
Late runtime: 53.02 seconds, 9.4 steps, 5.64 seconds per step \
Early Marginal Likelihood, Optimal: -583.55, Estimated: -572.67, Initial -646.06 \
Late Marginal Likelihood, Optimal: -589.58, Estimated: -569.92, Initial -744.68
| method     |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:-----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference |   1.3934  |      0.440782 |     0.10789   |  2.67044 |      5.93286 |     0.497026 | 0.488744 |
| initial    |   2.88641 |      0.521309 |     0.195614  |  3.11633 |      1.21088 |     0.180157 | 0.201638 |
| late_em    |   1.39654 |      0.431537 |     0.0970151 |  3.62865 |      3.70721 |     0.212556 | 0.238202 |
| real_early | nan       |    nan        |   nan         |  1.88592 |      1.78127 |     0.155295 | 0.287462 |
####

------------------------------------
##### Results for Additional real-early baseline
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 3.9 seconds, 7.8 steps, 0.5 seconds per step \
Late runtime: 47.23 seconds, 9.2 steps, 5.13 seconds per step \
Early Marginal Likelihood, Optimal: -567.07, Estimated: -554.35, Initial -643.82 \
Late Marginal Likelihood, Optimal: -606.14, Estimated: -580.97, Initial -732.21
| method     |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:-----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference |   1.27303 |      0.816239 |      0.113363 |  2.59347 |      4.81423 |     0.403711 | 0.570877 |
| initial    |   2.98315 |      0.565456 |      0.191513 |  2.77793 |      1.33573 |     0.162804 | 0.321497 |
| late_em    |   1.18451 |      0.822633 |      0.111398 |  2.68928 |      2.2549  |     0.160683 | 0.309943 |
| real_early | nan       |    nan        |    nan        |  2.16151 |      1.63829 |     0.171389 | 0.334537 |
####

------------------------------------
##### Results for Fixed predicting latent traits
Latent dimension: 2.0,  Item dimension: 12.0, sample size 100.0 \
Early runtime: 4.24 seconds, 8.6 steps, 0.49 seconds per step \
Late runtime: 42.91 seconds, 8.8 steps, 4.88 seconds per step \
Early Marginal Likelihood, Optimal: -542.31, Estimated: -535.07, Initial -624.32 \
Late Marginal Likelihood, Optimal: -616.66, Estimated: -609.14, Initial -779.98
| method     |   early_A |   early_delta |   early_sigma |   late_A |   late_delta |   late_sigma |      psi |
|:-----------|----------:|--------------:|--------------:|---------:|-------------:|-------------:|---------:|
| difference |   1.31683 |      0.727365 |     0.0819637 |  2.81163 |      6.04203 |    0.0979097 | 0.607177 |
| initial    |   3.17144 |      0.593239 |     0.208215  |  2.84086 |      1.19559 |    0.201808  | 0.299464 |
| late_em    |   1.38044 |      0.681315 |     0.167655  |  2.30619 |      2.3331  |    0.298597  | 0.228804 |
| real_early | nan       |    nan        |   nan         |  2.82787 |      2.31606 |    0.162536  | 0.307249 |
####
Performance on Individual Level \
rmse theta: 0.5383205034871745 \
rmse gain: 0.8237588915191916

### Seperated Q-Matrix

In [ ]:
experiment_0 = lambda sample_size, item_dimension, latent_dimension: mirt_simulation_experiment(
                                             sample_size=sample_size, item_dimension=item_dimension, latent_dimension=latent_dimension,
                                             q_type="seperated", stop_threshold=0.01,
                                             early_person_method="BFGS",
                                             late_person_method="BFGS",
                                            sigma_constraint="early_constraint",
                                            methods=["late_em", "real_early", "pure_competency", "initial", "difference"])
result_df, errors= repeat_mirt_experiment(experiment_0, repetitions=10, sample_sizes=[30, 100, 200], latent_dims=[2, 3], item_dims=[10, 20], file="results/seperated_q.csv")

Real latent covariance: [[ 1.      0.6393  0.2569 -0.1626]
 [ 0.6393  1.     -0.1999 -0.6009]
 [ 0.2569 -0.1999  0.5     0.4075]
 [-0.1626 -0.6009  0.4075  0.8321]]
Start late EM
Covariance matrix is good: True
EM Iteration 2
Current Monte Carlo Sample size: 300


/home/jesper/anaconda3/lib/python3.9/site-packages/scipy/stats/_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 12.601013312886394, current marginal loglikelihood: -318.7396548928781
EM Iteration 3
Current Monte Carlo Sample size: 300
Step: 3: current parameter_diff: 5.998677742873506, current marginal loglikelihood: -316.0539098100493
EM Iteration 4
Current Monte Carlo Sample size: 324


/home/jesper/anaconda3/lib/python3.9/site-packages/scipy/stats/_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 4: current parameter_diff: 4.186941825706572, current marginal loglikelihood: -314.8798063308089
EM Iteration 5
Current Monte Carlo Sample size: 324


...

Current Monte Carlo Sample size: 600
Step: 4: current parameter_diff: 3.732637347951136, current marginal loglikelihood: -2863.840862432545
EM Iteration 5


/home/jesper/anaconda3/lib/python3.9/site-packages/scipy/stats/_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 648
Step: 5: current parameter_diff: 2.6303992716977853, current marginal loglikelihood: -2863.996798119765
Absolute diff in A:
[[0.06602555 0.         0.        ]
 [0.4787442  0.         0.        ]
 [0.1191601  0.         0.        ]
 [0.30740235 0.         0.        ]
 [0.34781323 0.         0.        ]
 [0.08517905 0.         0.        ]
 [0.21262348 0.         0.        ]
 [0.01715095 0.         0.        ]
 [0.3898092  0.         0.        ]
 [0.00438416 0.         0.        ]
 [0.         0.35379386 0.        ]
 [0.         0.22054517 0.        ]
 [0.         0.01592691 0.        ]
 [0.         0.         0.04737592]
 [0.         0.         0.16737367]
 [0.         0.         0.28067107]
 [0.         0.         0.16185604]
 [0.         0.         0.75244033]
 [0.         0.         0.01550908]
 [0.         0.         0.03674342]]
Absolute diff in delta:
[0.24372655 0.05387496 0.03092906 0.39650915 0.25589298 0.03891264
 0.1253036  0.12574124 0.24

/home/jesper/anaconda3/lib/python3.9/site-packages/scipy/stats/_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Step: 2: current parameter_diff: 7.099639524649334, current marginal loglikelihood: -2192.8827189428057
EM Iteration 3
Current Monte Carlo Sample size: 648
Step: 3: current parameter_diff: 3.3423616077036638, current marginal loglikelihood: -2178.7710252857237
EM Iteration 4
Current Monte Carlo Sample size: 699
Step: 4: current parameter_diff: 2.193231708899176, current marginal loglikelihood: -2175.155080307205
EM Iteration 5
Current Monte Carlo Sample size: 754
Step: 5: current parameter_diff: 1.534593263858215, current marginal loglikelihood: -2172.2132562252395
EM Iteration 6
Current Monte Carlo Sample size: 814
Step: 6: current parameter_diff: 1.634779259009866, current marginal loglikelihood: -2169.796367763457
[[1.  0.  0.  0.5 0.5 0.5]
 [0.  1.  0.  0.5 0.5 0.5]
 [0.  0.  1.  0.5 0.5 0.5]
 [0.5 0.5 0.5 1.  0.5 0.5]
 [0.5 0.5 0.5 0.5 1.  0.5]
 [0.5 0.5 0.5 0.5 0.5 1. ]]
EM Iteration 2
Current Monte Carlo Sample size: 600
Step: 2: current parameter_diff: 23.3651640988193, current

### Pyramid Q-Matrix

In [ ]:
experiment_0_3 = lambda sample_size, item_dimension, latent_dimension: mirt_simulation_experiment(
                                             sample_size=sample_size, item_dimension=item_dimension, latent_dimension=latent_dimension,
                                             q_type="pyramid", stop_threshold=0.01,
                                             ensure_id=True,
                                             early_person_method="BFGS",
                                             late_person_method="BFGS",
                                            sigma_constraint="early_constraint",
                                            methods=["late_em", "real_early", "pure_competency", "initial", "difference"])
result_df, errors= repeat_mirt_experiment(experiment_0_3, repetitions=10, sample_sizes=[30, 100, 200], latent_dims=[2, 3], item_dims=[10, 20], file="results/pyramid_q.csv")

------------------------------------
##### Results for Seperated Q-Matrix
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 142.07 seconds, 26 steps, 5.46 seconds per step \
Optimal marginal Likelihood: -9384.38, Estimated: -10189.47, Initial -11459.9
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.00013 |      3.10678 |    0.0201383 |
| early_initial |  4.5183  |      2.5846  |    0.165491  |

------------------------------------
##### Results for BFGS for Q_0 with approximate Jacobi
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 61.55 seconds, 12 steps, 5.13 seconds per step \
Optimal marginal Likelihood: -9363.89, Estimated: -9711.58, Initial -10528.79
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.02507 |      1.83482 |    0.0645047 |
| early_initial |  2.06432 |      2.18337 |    0.289049  |

------------------------------------
##### Results for Monte Carlo sample varying through ttest
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 325.73 seconds, 42 steps, 7.76 seconds per step \
Optimal marginal Likelihood: -9929.77, Estimated: -10179.96, Initial -10767.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.25777 |     0.965894 |   0.00557421 |
| early_initial |  1.98812 |     1.68516  |   0.719705   |

------------------------------------
##### Results for Fixed random Covariance 1, girth included
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 57.16 seconds, 14 steps, 4.08 seconds per step \
Optimal marginal Likelihood: -11668.66, Estimated: -11672.6, Initial -12307.41
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.183251 |    0.0861252 |    0.0215108 |
| early_initial | 0.786712 |    0.393667  |    0.0471788 |
| girth         | 1.28298  |    0.263374  |    0.350541  |

------------------------------------
##### Results for Fixed Issue with simulation Covariance
Latent dimension: 2,  Item dimension: 20, sample size 1000 \
Runtime: 24.71 seconds, 11 steps, 2.25 seconds per step \
Optimal marginal Likelihood: -11186.38, Estimated: -11389.57, Initial -12010.77
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  1.83766 |      1.4664  |    0.0743169 |
| early_initial |  2.7041  |      2.66224 |    0.0627911 |

In [ ]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=20, latent_dimension=3, q_type="pyramid", girth=False, stop_threshold=10, ensure_id=True, q_share=0.6)

Real latent covariance: [[1.     0.5392 0.4093]
 [0.5392 1.     0.5406]
 [0.4093 0.5406 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:198: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))


[[1.         0.22255116 0.4851635 ]
 [0.22255116 1.         0.28710379]
 [0.4851635  0.28710379 1.        ]]
Step: 2: current parameter_diff: 22.836966115835242, current marginal loglikelihood: -11263.404209075177
EM Iteration 3


...

c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 877
[[ 1.         -0.18130942  0.49736388]
 [-0.18130942  1.         -0.04688012]
 [ 0.49736388 -0.04688012  1.        ]]
Step: 7: current parameter_diff: 4.21590743744123, current marginal loglikelihood: -10830.644137405307
EM Iteration 8


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 964
[[ 1.         -0.15152568  0.50240847]
 [-0.15152568  1.         -0.07665482]
 [ 0.50240847 -0.07665482  1.        ]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


In [ ]:
print_result(performance_dict, "With Covariance Matrix partially aligned to Q-matrix")

NameError: name 'performance_dict' is not defined

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 200 \
Runtime: 121.22 seconds, 46 steps, 2.64 seconds per step \
Optimal marginal Likelihood: -2144.15, Estimated: -2155.08, Initial -2699.63
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  11.0034 |      1.77461 |     0.889828 |
| early_initial |  10.5116 |      2.15986 |     0.522199 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 313.22 seconds, 51 steps, 6.14 seconds per step \
Optimal marginal Likelihood: -9452.24, Estimated: -9813.87, Initial -11216.48
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.34115 |      1.4068  |     0.425982 |
| early_initial |  5.98238 |      2.25109 |     0.234288 |

------------------------------------
##### Results for Pyramid Q-Matrix
Latent dimension: 3,  Item dimension: 30, sample size 2000 \
Runtime: 1521.51 seconds, 76 steps, 20.02 seconds per step \
Optimal marginal Likelihood: -30844.13, Estimated: -32521.34, Initial -37390.6
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.55959 |       1.9952 |    0.314331  |
| early_initial |  3.32034 |       3.514  |    0.0239562 |

------------------------------------
##### Results for BFGS for Q_0 with approximated gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 350.28 seconds, 64 steps, 5.47 seconds per step \
Optimal marginal Likelihood: -10498.87, Estimated: -11303.0, Initial -12740.14
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  10.2704 |     0.68597  |    0.318713  |
| early_initial |  10.4626 |     0.846172 |    0.0895367 |

------------------------------------
##### Results for BFGS for Q_0 with exact gradient
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 152.91 seconds, 31 steps, 4.93 seconds per step \
Optimal marginal Likelihood: -11106.69, Estimated: -11211.51, Initial -12370.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  3.5074  |      2.54509 |     0.21123  |
| early_initial |  3.51819 |      2.66313 |     0.319647 |

------------------------------------
##### Results for Newton-Raphson for Q_0 with approximated second derivative
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 352.93 seconds, 62 steps, 5.69 seconds per step \
Optimal marginal Likelihood: -10237.22, Estimated: -10482.36, Initial -12569.49
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  5.23694 |      2.45127 |     0.508499 |
| early_initial |  5.36172 |      3.29141 |     0.640209 |

In [ ]:
performance_dict = mirt_param_recovery(sample_size=1000, item_dimension=30, latent_dimension=3, q_type="full", girth=False, stop_threshold=10, q_share=0.0)

Real latent covariance: [[1.     0.     0.2545]
 [0.     1.     0.3721]
 [0.2545 0.3721 1.    ]]
Covariance matrix is good: True
EM Iteration 2


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\stats\_qmc.py:2039: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples = self.engine.random(n)


Current Monte Carlo Sample size: 600


c:\Users\Jesper\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 2: current parameter_diff: 41.855085339761395, current marginal loglikelihood: -12550.72345228223
EM Iteration 3
Current Monte Carlo Sample size: 660
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Invalid Covariance encountered, trying last step covariance
Step: 3: current parameter_diff: 28.922785136715092, current marginal loglikelihood: -12131.893852917503
EM Iteration 4
Current Monte Carlo Sample size: 660
[[ 1.00000000e+000 -7.56483606e-249 -7.12199634e-243]
 [-7.56483606e-249  1.00000000e+000 -1.46994029e-260]
 [-7.12199634e-243 -1.46994029e-260  1.00000000e+000]]


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: divide by zero encountered in log
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))
c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:264: RuntimeWarning: invalid value encountered in multiply
  icc_values), r_item_theta) + np.multiply(np.log(1-icc_values), np.subtract(r_0_theta, r_item_theta))


Step: 4: current parameter_diff: 24.909956480277785, current marginal loglikelihood: -11869.365012663717
EM Iteration 5
Current Monte Carlo Sample size: 660
[[1.         0.07473132 0.19983603]
 [0.07473132 1.         0.24058055]
 [0.19983603 0.24058055 1.        ]]
Step: 5: current parameter_diff: 20.731814136925763, current marginal loglikelihood: -11739.105149945846
EM Iteration 6
Current Monte Carlo Sample size: 660
[[1.         0.02395303 0.25925938]
 [0.02395303 1.         0.26711545]
 [0.25925938 0.26711545 1.        ]]
Step: 6: current parameter_diff: 6.423400003338906, current marginal loglikelihood: -11720.71525563623
EM Iteration 7
Current Monte Carlo Sample size: 726
[[1.         0.02766565 0.2770886 ]
 [0.02766565 1.         0.30801604]
 [0.2770886  0.30801604 1.        ]]
Step: 7: current parameter_diff: 5.599935651052099, current marginal loglikelihood: -11726.261698793634
EM Iteration 8
Current Monte Carlo Sample size: 798
[[1.         0.03611896 0.29298871]
 [0.03611896

c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\m_step_mirt_2pl.py:70: RuntimeWarning: overflow encountered in exp
  population = random.choices(population=population_base, weights=np.exp(


Step: 8: current parameter_diff: 8.88763105930743, current marginal loglikelihood: -11739.130271363902
EM Iteration 9
Current Monte Carlo Sample size: 877
[[1.         0.04450944 0.2860444 ]
 [0.04450944 1.         0.34325432]
 [0.2860444  0.34325432 1.        ]]
Step: 9: current parameter_diff: 6.972349751947515, current marginal loglikelihood: -11721.612152810394
EM Iteration 10
Current Monte Carlo Sample size: 964
[[1.         0.04267512 0.28987184]
 [0.04267512 1.         0.33339445]
 [0.28987184 0.33339445 1.        ]]
Step: 10: current parameter_diff: 13.349635834760017, current marginal loglikelihood: -11712.217814439908
EM Iteration 11
Current Monte Carlo Sample size: 1060
[[1.         0.05402791 0.27611641]
 [0.05402791 1.         0.32730623]
 [0.27611641 0.32730623 1.        ]]
Step: 11: current parameter_diff: 13.121900001706786, current marginal loglikelihood: -11752.478352914124
EM Iteration 12
Current Monte Carlo Sample size: 1166
[[1.         0.03472167 0.29289335]
 [0.0

KeyboardInterrupt: 

In [ ]:
print_result(performance_dict, "With fixed Discrimination sampling and BFGS")

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 63.64 seconds, 14 steps, 4.55 seconds per step \
Optimal marginal Likelihood: -8684.81, Estimated: -8676.2, Initial -9809.83
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     | 0.933735 |     0.514423 |    0.0383988 |
| early_initial | 2.67536  |     0.636687 |    0.231996  |


------------------------------------
##### Results for With BFGS as covariance optimization Algorithm
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 160.7 seconds, 20 steps, 8.04 seconds per step \
Optimal marginal Likelihood: -11451.44, Estimated: -11746.37, Initial -12757.23
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  4.12413 |      1.19297 |     0.539132 |
| early_initial |  4.06187 |      1.22103 |     0.159225 |

------------------------------------
##### Results for With Cholesky covariance decomposition
Latent dimension: 3,  Item dimension: 20, sample size 1000 \
Runtime: 60.82 seconds, 14 steps, 4.34 seconds per step \
Optimal marginal Likelihood: -11369.92, Estimated: -12102.54, Initial -13629.98
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  2.47435 |     0.400838 |     0.181127 |
| early_initial |  2.98819 |     0.686696 |     0.142997 |

------------------------------------
##### Results for With Cholesky covariance decomposition and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 94.77 seconds, 12 steps, 7.9 seconds per step \
Optimal marginal Likelihood: -15960.47, Estimated: -17344.15, Initial -19786.45
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  48.8803 |     0.777924 |     0.149531 |
| early_initial |  49.0831 |     1.33755  |     0.207159 |

------------------------------------
##### Results for With fixed Discrimination sampling and BFGS
Latent dimension: 3,  Item dimension: 30, sample size 1000 \
Runtime: 91.68 seconds, 12 steps, 7.64 seconds per step \
Optimal marginal Likelihood: -12149.74, Estimated: -12157.89, Initial -14965.74
|               |   rmse_A |   rmse_delta |   rmse_sigma |
|:--------------|---------:|-------------:|-------------:|
| estimated     |  0.3929  |     0.249323 |    0.0275205 |
| early_initial |  2.20296 |     0.582838 |    0.380006  |

## Experiment 2: Competency Gain Prediction